# SLEAP Feature Extraction for Home Cage

Features that will be extracted:
- head distance
- res head-int head angle
- res head-int hind distance
- res head-int hind angle
- int head-res hind distance
- int head-res hind angle
- hind distance
- resident velocity
- inrtuder velocity

Need to grab when a intruder is in and the identity of the intruder for downstream analysis

Extra Features to include:
- Brain region
- mouse identity
- standard zscored DA
- Intuder social identity

Convert h5 file data to only include times when another intruder is in

Convert frames to time to be able to line up DA values to times

Extract features from frames

In [2]:
import os
import sys

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(PROJECT_ROOT)

import h5py
import numpy as np
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter
from bouts_extension import *
from slp_trl_class import SleapTrial

In [3]:
file = r"C:\Users\alber\OneDrive\Desktop\PC_Lab\Photometry\Pilot_2\Combined_Cohorts\Home_Cage\nac\nn2-250120-072910"
csv_path = r"C:\Users\alber\OneDrive\Desktop\PC_Lab\Photometry\Pilot_2\Combined_Cohorts\Home_Cage\nac_csvs\nn2-250120-072910.csv"

In [4]:
trial = SleapTrial(file, '_465A', '_405A')

Found Synapse note file: C:\Users\alber\OneDrive\Desktop\PC_Lab\Photometry\Pilot_2\Combined_Cohorts\Home_Cage\nac\nn2-250120-072910\Notes.txt
read from t=0s to t=1774.16s


In [5]:
bout_definitions = [
    {'prefix': 'Short_Term', 'introduced': 'Short_Term_Introduced', 'removed': 'Short_Term_Removed'},
    {'prefix': 'Long_Term', 'introduced': 'Long_Term_Introduced', 'removed': 'Long_Term_Removed'},
    {'prefix': 'Novel', 'introduced': 'Novel_Introduced', 'removed': 'Novel_Removed'}
]
trial.extract_bouts_and_behaviors(csv_path, bout_definitions, first_only=False)
trial.combine_consecutive_behaviors(behavior_name='all', bout_time_threshold=1)

In [6]:
trial.downsample(target_fs = 100)

# 2.5) Remove beginning and end
trial.remove_initial_LED_artifact(t=30)
trial.remove_final_data_segment(t=10)

# 3) low‐pass
trial.lowpass_filter(cutoff_hz=3.0)

# 4) high‐pass recentered
trial.baseline_drift_highpass_recentered(cutoff=0.001)

# 5) IRLS fit
trial.motion_correction_align_channels_IRLS(IRLS_constant=1.4)

# 6) compute dF/F
trial.compute_dFF()

# 7) zscore
trial.compute_zscore(method='standard')

Sleap

In [7]:
filename = r"C:\Users\alber\OneDrive\Desktop\PC_Lab\Photometry\Pilot_2\Combined_Cohorts\Home_Cage\SLEAP\sleap_id_corrected\1_19_25_Home_Cage_C3-250118-111408_nn2-250120-072910_Cam1_converted.analysis.h5"
trial.load_sleap(h5_path=filename, fps = 10)

In [8]:
trial.filter_sleap_bouts(interp_kind="linear")

In [9]:
trial.smooth_locations()

In [16]:
trial.node_dict

{'Head': 0,
 'Right_Ear': 1,
 'Nose': 2,
 'Tail_Base': 3,
 'Left_Lateral': 4,
 'Right_Lateral': 5,
 'Left_Ear': 6,
 'Neck': 7,
 'Center': 8}

In [10]:
trial.track_dict

{'subject': 0, 'agent': 1}

Adding DA and metadata

In [11]:
trial.add_metadata_and_DA()

,time_s,brain_region,mouse_identity,zscore_DA,intruder_identity
0,180.6,NAc,nn2,-0.754077,Short_Term_1
1,180.7,NAc,nn2,-0.538230,Short_Term_1
2,180.8,NAc,nn2,-0.203720,Short_Term_1
3,180.9,NAc,nn2,-0.036774,Short_Term_1
4,181.0,NAc,nn2,0.050408,Short_Term_1
...,...,...,...,...,...
15041,1773.5,NAc,nn2,0.648638,Novel_1
15042,1773.6,NAc,nn2,0.648638,Novel_1
15043,1773.7,NAc,nn2,0.648638,Novel_1
15044,1773.8,NAc,nn2,0.648638,Novel_1


In [12]:
trial.compute_pairwise_features()

,time_s,brain_region,mouse_identity,zscore_DA,intruder_identity,head_dist,hind_dist,res_head_int_hind_dist,res_head_int_hind_angle,int_head_res_hind_dist,int_head_res_hind_angle,resident_velocity,intruder_velocity
0,180.6,NAc,nn2,-0.754077,Short_Term_1,282.648696,286.772588,265.808599,-0.027132,277.586537,3.114768,43.988459,71.589673
1,180.7,NAc,nn2,-0.538230,Short_Term_1,270.555166,287.805233,270.524078,-0.053932,265.749896,3.064875,40.681429,71.647760
2,180.8,NAc,nn2,-0.203720,Short_Term_1,258.432852,287.974587,272.912598,-0.080019,254.663420,3.022936,34.305577,73.370442
3,180.9,NAc,nn2,-0.036774,Short_Term_1,246.284834,287.076529,273.191862,-0.105853,244.042928,2.989251,28.405654,76.452938
4,181.0,NAc,nn2,0.050408,Short_Term_1,234.129310,285.006829,271.582236,-0.131789,233.691134,2.964000,22.981901,80.192820
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15041,1773.5,NAc,nn2,0.648638,Novel_1,209.766192,110.760377,160.938603,2.015316,180.249534,-0.424718,2.569354,92.706046
15042,1773.6,NAc,nn2,0.648638,Novel_1,215.219535,119.768548,166.849207,2.073099,187.911678,-0.396345,4.447339,87.009530
15043,1773.7,NAc,nn2,0.648638,Novel_1,220.192223,128.854941,172.856455,2.134949,194.555431,-0.370061,7.043566,80.934785
15044,1773.8,NAc,nn2,0.648638,Novel_1,224.570424,138.045860,179.072800,2.201045,200.024344,-0.345181,10.090624,74.971632


DA

In [13]:
# trial.downsample(target_fs = 100)
# # 2.5) Remove beginning and end
# trial.remove_initial_LED_artifact(t=30)
# trial.remove_final_data_segment(t=10)
# # 3) low‐pass
# trial.lowpass_filter(cutoff_hz=3.0)
# # 4) high‐pass recentered
# trial.baseline_drift_highpass_recentered(cutoff=0.001)
# # 5) IRLS fit
# trial.motion_correction_align_channels_IRLS(IRLS_constant=1.4)
# # 6) compute dF/F
# trial.compute_dFF()
# # 7) zscore
# trial.compute_zscore(method='standard')

Sanity Checks - make sure calculations are correct

In [14]:
import numpy as np

# parameters
fps = 10.0
F   = 50
d   = 42.0

# make a dummy SleapTrial‐like object
class Dummy:
    pass

t = Dummy()
t.frame_times   = np.arange(F)/fps
# shape = (frames, nodes, dims, instances)
# here we only need 1 node, 2 instances
t.locations = np.zeros((F, 1, 2, 2))
for f in range(F):
    t.locations[f,0,0,0] = 0    # subj x
    t.locations[f,0,1,0] = 0    # subj y
    t.locations[f,0,0,1] = d    # agn x
    t.locations[f,0,1,1] = 0    # agn y

t.node_dict  = {'nose':0}
t.track_dict = {'subject':0, 'agent':1}

# now call your distance_between
dist = SleapTrial.distance_between(t,
           node1='nose', track1='subject',
           node2='nose', track2='agent')

print("mean dist:", np.mean(dist))
print("expected :", d)


mean dist: 42.0
expected : 42.0


In [15]:
import numpy as np

# parameters
fps   = 10.0
F     = 100                 # 10 s of data
v_true = np.array([2.0,3.0])  # px/s

# build a dummy SleapTrial-like object
class Dummy:
    pass

t = Dummy()
t.frame_times = np.arange(F)/fps

# monkey‐patch a single node, single instance into t.locations
# shape must be (frames, nodes, dims, instances)
t.locations = np.zeros((F, 1, 2, 1))
for f in range(F):
    t.locations[f,0,:,0] = v_true * t.frame_times[f]

# build node_dict so our routine finds it
t.node_dict = {'dummy':0}
t.track_dict = {'inst':0}

# plug in the node_velocity from above
def node_velocity(t, node):
    idx    = t.node_dict[node]
    coords = t.locations[:, idx, :, :]       # (F,2,1)
    dt     = np.mean(np.diff(t.frame_times))
    out    = []
    for i in range(coords.shape[-1]):
        xy = coords[...,i]                   # (F,2)
        dx = np.gradient(xy[:,0], dt)
        dy = np.gradient(xy[:,1], dt)
        out.append(np.sqrt(dx*dx + dy*dy))
    return np.stack(out,0)

vel = node_velocity(t, 'dummy')  # shape (1,F)

print("mean computed speed:", vel.mean())
print("expected speed     :", np.linalg.norm(v_true))
# should be within a few 1e‑3 pct or so


mean computed speed: 3.6055512754639887
expected speed     : 3.605551275463989


Understanding h5 files